https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("whitegrid")

notebooks_path = '../nbs/'
input_path = '../input/'
output_path = '../output/'
print(os.listdir(input_path))


# Any results you write to the current directory are saved as output.

## Loading data set files


In [ ]:
train_dataset = pd.read_csv(input_path + 'train.csv')
test_dataset = pd.read_csv(input_path + 'test.csv')
print('Train ({} rows), and test ({} rows) data sets were loaded'.format(train_dataset.shape[0], test_dataset.shape[0]))

## Data Exploration
Let's check the size, types  of the train and test data sets.
* Data sets size - how many rows and columns each data set contains
* What are the data types
* Missing values - how many where they are


In [ ]:
train_dataset.head(3)

## Counting how many columns from each data type we have:

In [ ]:
# Return a table with a count of all columns types 
def explore_dtypes(df):
    types = pd.DataFrame(df.dtypes, columns=['Column Data Type'])
    types['Count'] = types.index
    types.reset_index(drop=True, inplace=True)
    return types.groupby(by=['Column Data Type']).count()
    
print('Columns count by data type\n\nTrain set:')
display(explore_dtypes(train_dataset))    

print('\n*************\nTest set:')
display(explore_dtypes(test_dataset))

* The data contains only columns of types **int64** and **float64**.
* Train and test sets data types look the same*. 

 \*The only diffence is that the train set contains 1 extra column, as expectedwhich is the **'target'** column (labels, type int64).

In [ ]:
diff_cols = [col for col in train_dataset.columns if col not in test_dataset.columns]
print ('Columns that in the train set and not in the test set:', diff_cols)


In [ ]:
display('Train', train_dataset.describe())
display('-------\n\nTest', test_dataset.describe())

## Exploring Missin Values
### Basic exploration of which columns have missing data and in which ratio.
Will be usfull for feature exploration and data cleaning.
* Replacing -1 with np.nan (Null)
 * missing values represented as '-1' in all the columns in the data set (taken from the official data description [link][http://sss.sss]).
 * Need to be replaced to np.nan in order to use pandas built in functions.
* Counting missing values by column.
* Displaying count and ratio of missing values of each column that has missing values.
* Displaying the ratio of missing values for the intire data set.

### Conclusions
Both train set and test set have simmiller total missing value ratio.

~2.4% of the data is missing in each of them.

Both train set and test set have missing values in the same columns* and with very simmiller ratio per column.

*There is one column,'ps_car_12' , that contains only one missing value, and only in the train set.

Some columns only have a small amount of missing values while others have high ratio of nulls.

The two most "problematic" columns are **'ps_car_o3_cat'**(~69%) and **'ps_car_05_cat'**(~44%) in both train and test sets.


------
# remove
There are 59 columns, 13 of them have missing values (train set).

There are 58 columns, 12 of them have missing values (test set).




In [ ]:
from IPython.display import display, HTML
def explore_missing_values(ds):
    # Replacing '-1' with np.nan (null)
    ds.replace(-1, np.nan, inplace= True)

    # Counting mising values (nulls) for each column and displaying the columns orderd by nulls count
    rows = ds.shape[0] 
    columns = ds.shape[1]
    na_df = pd.DataFrame(ds.isnull().sum(), columns=['Nulls_count'])
    na_df['Nulls_ratio'] = np.round((na_df['Nulls_count'] / rows) * 100, 3)
    na_df.drop(na_df[na_df['Nulls_count'] == 0].index, axis=0, inplace=True)
    na_df.sort_values(by=['Nulls_count'], ascending=False, inplace=True)

    # Total missing values count and ratio
    nulls_count = ds.isnull().sum().sum()
    total_missing_ratio = np.round((nulls_count / (rows * columns)) * 100, 2)
    print('There are {} missing values ({}% of the data)'.format(nulls_count, total_missing_ratio))
    print('Out of {} columns, {} columns have missing values'.format(columns, len(na_df)))
    print('The column "{}" has the highest ratio of missing values (~{}%)'.format(na_df.index[0], na_df.iloc[0,1]))
    return na_df
print('Train set:')
nulls_df = explore_missing_values(train_dataset)
display(nulls_df)
print('****\n****\n\nTest set:')
display(explore_missing_values(test_dataset))

# Bar chart of training data
fig, ax = plt.subplots(figsize=(16,8))
ax.set_title('Null count - Train data')    

# Values that are less than 1% are rounded to 1% for better chart display
nulls_df.loc[nulls_df[nulls_df['Nulls_ratio']<1].index, ['Nulls_ratio']] = 1 
sns.barplot(x=nulls_df.index, y=nulls_df['Nulls_ratio'], ax=ax)
#ax.set_ylabel('Unique values count')

## Outlier detection and removal


The data contains numerical values, categriacal features and binary features.

The type of the feature can be detected by the column name prefix (from data description[link][0])

In [ ]:
features = train_dataset.columns[2:]
categorical_features = [col for col in features if '_cat' in col]
binary_features = [col for col in features if '_bin' in col]
numeric_features = [col for col in features if '_cat' not in col and '_bin' not in col]

print('{} Categorical features:\n{}'.format(len(categorical_features),categorical_features))
print('\n{} Binary features:\n{}'.format(len(binary_features), binary_features))
print('\n{} Numeric features:\n{}'.format(len(numeric_features), numeric_features))

## Outlier detection - Need to check if processing it after filling missing values (nulls) gives better results

In [ ]:
# Outlier detection 
from collections import Counter

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v >= n )
    
    print('There are {0} rows ({2:.2f}%) that contains at least {1} outliers each'.format(len(multiple_outliers), n,100 * len(multiple_outliers)/len(df)))
    return multiple_outliers   

# detect outliers in numeric_features
Outliers_to_drop = detect_outliers(train_dataset.dropna(axis=0), 6,numeric_features)
display(train_dataset.ix[Outliers_to_drop, numeric_features])
display(train_dataset[numeric_features].describe())


## Feature analysis
### Numerical values


In [ ]:
# Correlation matrix between numerical values (SibSp Parch Age and Fare values) and Survived 
corr_features = numeric_features[:10].copy()
corr_features.append('target')
fig, ax = plt.subplots(figsize=(12,12))
g = sns.heatmap(train_dataset[corr_features].corr(),annot=True, fmt = ".2f", cmap = "coolwarm", ax=ax)


In [ ]:
#a = train_dataset.corr()
#a.sort_values(by=['target'], ascending=False)

In [ ]:
def count_unique_values(col):
    return len(col.value_counts())

def get_feature_type(col):
    name = col.name
    if '_bin' in name or 'target' in name:
        f_type = 'binary'
    elif '_cat' in name:
        f_type = 'categorical'
    elif col.dtype == np.int64:
        f_type = 'ordinal'
    else:
        f_type = 'interval'
    return f_type

def get_feature_group(col):
    splited_name = col.name.split('_')
    if len(splited_name) > 1:
        group = splited_name[1]
    else:
        group = splited_name[0]
    return group

unique_values = train_dataset.apply(count_unique_values,axis=0)
unique_values = pd.DataFrame(unique_values, columns=['count'])
unique_values['d_type'] = train_dataset.dtypes
unique_values['f_type'] = train_dataset.apply(get_feature_type, axis=0, reduce=False)
unique_values['group'] = train_dataset.apply(get_feature_group, axis=0)
unique_values['missing_values'] = train_dataset.isnull().sum()
unique_values.drop(['id', 'target'], axis=0, inplace=True)


In [ ]:
unique_values.sort_values(by=['f_type', 'count', 'group', 'd_type', 'missing_values'])


In [ ]:
for f_type in ['ordinal', 'categorical', 'interval']:
    fig, ax = plt.subplots(figsize=(16,8))
    ax.set_title(f_type + ' features')    
    ax.set_xlabel('Name')
    sliced_data = unique_values[unique_values['f_type'] == f_type].reset_index()
    sns.set_style("whitegrid")
    sns.barplot(data=sliced_data, x='index', y='count', ax=ax)
    ax.set_ylabel('Unique values count')


##  A better look at features of type interval
The chart of these features is not so usefull as the unique values count is in range of 4 up to 70k.<br>In order to keep the chart informative, two extra steps are needed:
1. Split the data into 2 charts - features with low / high number of unique values.
2. Drop the highest value (ps_car_13 with 70482 unique values) so it is possible to notice the rest of the values.

In [ ]:
fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1, figsize=(16,8))
ax1.set_title('interval features')    
ax1.set_xlabel('Name')
sliced_data = unique_values[unique_values['f_type'] == 'interval'].reset_index().sort_values(by='count')
sliced_data_start = sliced_data[:-4]
sliced_data_end = sliced_data[-4:-1]
ax1.set_yticks(sliced_data_start['count'].values)
sns.set_style("whitegrid")
sns.barplot(data=sliced_data_start, x='index', y='count', ax=ax1)
ax1.set_ylabel('Unique values count')

ax2.set_yticks(sliced_data_end['count'].values)
sns.barplot(data=sliced_data_end, x='index', y='count', ax=ax2, )
ax2.set_ylabel('Unique values count')


In [ ]:
for f_type in unique_values['f_type'].unique():
    sliced_features = list(unique_values[unique_values['f_type'] == f_type].index)
    sliced_features.append('target')
    sliced_data = train_dataset[sliced_features]
    fig, ax = plt.subplots(figsize=(16,8))
    ax.set_title(f_type + ' features - corrolation')    
    sns.heatmap(data=sliced_data.corr(), ax=ax, annot=True, fmt = ".2f", cmap = "coolwarm", linewidths=1)

In [ ]:
unique_values_with_missing = unique_values[unique_values.missing_values > 0]
train_dataset[unique_values_with_missing.index]
fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(train_dataset[unique_values_with_missing.index].corr(), fmt='.2f', annot=True, linewidths=1, ax=ax)

corr = train_dataset.corr()
most_corr = {}
for col in unique_values_with_missing.index.values:
    sorted_corr = corr.sort_values(by=col, ascending=False)
    most_corr[col] = sorted_corr[col][1:5].reset_index().values
#display(most_corr)

In [ ]:
corr_df = pd.DataFrame(data={'Name':list(most_corr.keys())})
#corr_df['']
vals = np.array(list(most_corr.values()))
vals[:,1,:]
#corr_df

In [ ]:
# Need to make a df from it with the top corrolated features for each feature that has missing values
# Next sstep is to fill the missing values according to the values of the same feature in rows where the corrolated other features are th smae

aa = np.array(list(most_corr.items()))[:,1]
aa[:]

In [ ]:
train_dataset['ps_car_03_cat'].value_counts()

In [ ]:
# Drop nulls - 
print('Data size with nulls', train_dataset.shape)
# Drop columns with more than 15% missing values

too_many_missing_values_cols = nulls_df[nulls_df.Nulls_ratio > 20].index.values
print('Features with more than 20% missing values', too_many_missing_values_cols)

train_dataset = train_dataset.drop(too_many_missing_values_cols, axis=1)
print('Data size after droppint columns with more than 20% missing values', train_dataset.shape)
# Dropping rows with missing values
train_dataset = train_dataset.dropna(axis=0)
print('Data size after droppint rows that contain missing values', train_dataset.shape)

In [ ]:
# Categorical features encoding
#from sklearn.preprocessing import OneHotEncoder#,LabelEncoder
#lbl = LabelEncoder()
#ohe = OneHotEncoder()

#print(categorical_features)
#ohe.fit(train_dataset[['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat']])
#ohe.transform(train_dataset[['ps_ind_02_cat']])
#display(ohe.transform(train_dataset[['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat']]))
#train_dataset[['ps_ind_02_cat']].get_dummies()
#a = train_dataset['ps_calc_01']
categorical_features = [f for f in train_dataset.columns if '_cat' in f]
train_dataset = pd.get_dummies(train_dataset, columns=categorical_features, prefix=categorical_features)
#train_dataset[categorical_features]

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
X = train_dataset[train_dataset.columns[2:]]
y = train_dataset[train_dataset.columns[1]]
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=12, shuffle=True)
y_train.value_counts(), y_validation.value_counts(), X_train.shape

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

weights = compute_class_weight('balanced', y.unique(), y)
scale_pos_weight = weights[1]/weights[0]
scale_pos_weight, weights

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(objective='binary:logistic', n_jobs=-1, learning_rate=0.2, gamma=0.007 ,
                    n_estimators=30, min_child_weight=5,
                    max_depth=3, scale_pos_weight=scale_pos_weight,# subsample=0.8,colsample_bytree=0.8,
                    verbose=2)#, eval_metric=['xgb_normalizedgini', 'logloss'])


In [ ]:
xgb.fit(X_train, y_train ,eval_metric=gini_xgb)
print('Finished training')


In [ ]:
y_pred = xgb.predict(X_validation)
y_pred_prob = xgb.predict_proba(X_validation)


y_pred_train = xgb.predict(X_train)
y_pred_prob_train = xgb.predict_proba(X_train)

In [ ]:
"""from xgboost import XGBClassifier
xgb = XGBClassifier(objective='binary:logistic', n_jobs=-1, learning_rate=0.02, gamma=0.007 ,
                    n_estimators=100, min_child_weight=5, silent=False, 
                    max_depth=3, scale_pos_weight=scale_pos_weight,# subsample=0.8,colsample_bytree=0.8,
                    verbose=2, eval_metric=['auc'])
xgb.fit(X_train, y_train, eval_metric=gini_xgb)
print('Finished training')
y_pred = xgb.predict(X_validation)
y_pred_prob = xgb.predict_proba(X_validation)
y_pred_train = xgb.predict(X_train)
y_pred_prob_train = xgb.predict_proba(X_train)"""

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
display(confusion_matrix(y_train, y_pred_train))
display(classification_report(y_train, y_pred_train))

display(confusion_matrix(y_validation, y_pred))
display(classification_report(y_validation, y_pred))
print(eval_gini(y_validation, y_pred_prob[:, 1]))
print(eval_gini(y_train, y_pred_prob_train[:, 1]))

In [ ]:
pd.DataFrame(data={'Feature':X_train.columns.values, 'Importance':xgb.feature_importances_}).sort_values(by=['Importance'], ascending=False)

In [ ]:
feature_importances = pd.DataFrame(data={'Feature':X_train.columns.values, 'Importance':xgb.feature_importances_}).sort_values(by=['Importance'], ascending=False)
features_to_drop = feature_importances[feature_importances['Importance'] < 0.01]['Feature']
features_to_drop
train_dataset.drop(features_to_drop.values, axis=1, inplace=True)

In [ ]:
pd.DataFrame(data={'Feature':X_train.columns.values, 'Importance':xgb.feature_importances_}).sort_values(by=['Importance'], ascending=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=30, max_depth=8, min_samples_leaf=4, 
                            max_features=0.2, n_jobs=-1, random_state=0, class_weight='balanced_subsample')
rf.fit(X_train, y_train)
print('Finished training')
y_pred = xgb.predict(X_validation)

y_pred_train = xgb.predict(X_train)

In [ ]:
display(confusion_matrix(y_train, y_pred_train))
display(classification_report(y_train, y_pred_train))

display(confusion_matrix(y_validation, y_pred))
display(classification_report(y_validation, y_pred))

In [ ]:
#! pip install numba
from numba import jit

# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [ ]:
# Funcitons from olivier's kernel
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]


In [ ]:
xgb = XGBClassifier(objective='binary:logistic',
                    n_jobs=-1)

In [ ]:
# Random search with K fold cross validation

# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'scale_pos_weight': [scale_pos_weight/10, scale_pos_weight/5, scale_pos_weight],
        'learning_rate':[0.01, 0.03, 0.05, 0.1, 0.2],
        'n_estimators': [30, 100, 230]
        }

In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, 
                                   param_distributions=params,
                                   n_iter=param_comb, scoring='roc_auc',
                                   n_jobs=-1,
                                   cv=skf.split(X,y),
                                   verbose=10, random_state=1001, fit_params={'eval_metric':gini_xgb})

# Here we go
#start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, y)
#timer(start_time) # timing ends here for "start_time" variabl

In [ ]:
random_search.best_estimator_

In [ ]:
xgb = random_search.best_estimator_